In [11]:
import pandas as pd
import numpy as np
import mysql.connector



In [12]:
def getConf():
    file1 = open('conf.txt', 'r')
    Lines = file1.readlines()
    print(Lines)
    user = Lines[0].split('#')[0]
    pwd = Lines[0].split('#')[1]
    db = Lines[0].split('#')[2]
    
    return user, pwd, db

getConf()


['iandryAtout#iandryAtout#AtoutData']


('iandryAtout', 'iandryAtout', 'AtoutData')

In [27]:
def getConnection(user, pwd, db):
    mydb = mysql.connector.connect(
    host="localhost",
    user=user,
    password=pwd,
    database=db
    )


    #cursor = mydb.cursor()


    return mydb

#getConnection(user, pwd, db)

In [46]:
def queryHotel(typeEtablissement):
    query = ('select distinct dc.dt, nom_region, accueil, nbChambre from HebergementInfo h, DateClassement dc, Localisation l '\
            'where h.dateId = dc.dateId and h.cp = l.code_postal and  `typeId` = (select typeId from TypeEtablissement where value = "'+typeEtablissement+'" ) and '\
            'h.dateId = dc.dateId ')
    return query

In [58]:
def getData(conn, typeEtablissement):
    #query = ('SELECT * FROM HebergementInfo where `typeId` = (select typeId from TypeEtablissement ' \
    #        'where value = "'+typeEtablissement+'" )')
    query = queryHotel(typeEtablissement)
    print(query)
    cursor = conn.cursor()

    cursor.execute(query)


    result = cursor.fetchall()

    num_fields = len(cursor.description)
    field = [i[0] for i in cursor.description]


    return np.array(field), np.array(result)

In [59]:
user, pwd, db = getConf()
conn = getConnection(user, pwd, db)

fields, data = getData(conn, "HÔTEL DE TOURISME")
print(fields)
print(len(data))
print(data[0:5])

['iandryAtout#iandryAtout#AtoutData']
select distinct dc.dt, nom_region, accueil, nbChambre from HebergementInfo h, DateClassement dc, Localisation l where h.dateId = dc.dateId and h.cp = l.code_postal and  `typeId` = (select typeId from TypeEtablissement where value = "HÔTEL DE TOURISME" ) and h.dateId = dc.dateId 
['dt' 'nom_region' 'accueil' 'nbChambre']
13085
[[datetime.date(2020, 7, 15) 'Auvergne-Rhône-Alpes' '21' '10']
 [datetime.date(2017, 10, 18) 'Auvergne-Rhône-Alpes' '75' '50']
 [datetime.date(2017, 7, 24) 'Auvergne-Rhône-Alpes' '64' '31']
 [datetime.date(2016, 5, 25) 'Auvergne-Rhône-Alpes' '19' '10']
 [datetime.date(2017, 9, 10) 'Auvergne-Rhône-Alpes' '69' '35']]


Combien de temps avant que les hotels doivent reclasser

In [67]:
from dateutil.relativedelta import relativedelta

from datetime import date

today = date.today()
print("Today's date:", today)

dataAudit = []
for i in data:
    """
    print(i[0])
    print(relativedelta(today, i[0]).years)
    """
    dataAudit.append( [ i[getIndex(fields, 'dt')], 5 - (relativedelta(today, i[0]).years) , i[getIndex(fields, 'nom_region')] ] )
    
print(len(dataAudit))
print(dataAudit[0][1])

dataAudit = np.array(dataAudit)
attente = dataAudit[:,1]
attente = np.array(attente)

att, ct = np.unique(attente, return_counts=True)

for i in zip(att, ct):
    print(i)



#difference_in_years = relativedelta(end_date, start_date).years

Today's date: 2022-01-25
13085
4
(-2, 15)
(-1, 561)
(0, 1307)
(1, 4928)
(2, 2054)
(3, 1696)
(4, 811)
(5, 1713)
